In [1]:
from tkinter import *
import PIL
from PIL import Image, ImageDraw , ImageTk
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pk
import numpy as np
import warnings 
warnings.filterwarnings("ignore")

In [2]:
rf = pk.load(open('Models/rf_clf.sav','rb'))

In [3]:
char_map = pd.read_csv('Data/mappings.txt' , delimiter=' ', header=None , index_col=0 , squeeze=True)
label_dictionary = {}
for index, label in enumerate(char_map):
    label_dictionary[index] = chr(label)

In [4]:
def save():
    filename = 'Images/img.png' 
    image1.save(filename)
        

In [5]:
def activate_paint(e):
    global lastx, lasty
    cv.bind('<B1-Motion>', paint)
    lastx, lasty = e.x, e.y

In [6]:
def paint(e):
    global lastx, lasty
    x, y = e.x, e.y
    cv.create_line((lastx, lasty, x, y),fill = 'white', width=20)
    #  --- PIL
    draw.line((lastx, lasty, x, y), fill='white', width=20)
    lastx, lasty = x, y

In [7]:
def clear():
    cv.delete("all")
    redraw()

In [8]:
def pred_img():
    global text_area , rotate
    pix = text_area.get(1.0 , END)
    pix = pix.split('\t')
    pix = np.asarray(pix)
    pix = rotate(pix)
    pix = pix.astype(float)
    pix = pix.reshape(-1)
    y_pred_samp = rf.predict([pix])

    label_pred.config(text = "Predicted: " + label_dictionary[( y_pred_samp[0] )] )    
    

In [9]:
def get_pred_dr():
    global pred_text
    im = cv2.imread('Images/img.png')
    final = resize_img(im)
    final = final.reshape(-1)
    y_pred_samp = rf.predict([final])
    label_pred.config(text = "Predicted: " + label_dictionary[( y_pred_samp[0] )] )

In [10]:
def resize_img(cimage):
    gray = cv2.cvtColor(cimage , cv2.COLOR_BGR2GRAY)
    resized_img = cv2.resize(gray , (28,28) , interpolation = cv2.INTER_AREA)
    return resized_img

In [11]:
def redraw():
    global image1 , draw
    image1 = PIL.Image.new('L', (140,140), 'black')
    draw = ImageDraw.Draw(image1)

In [12]:
def get_img_from_pixels():

    global text_area , im , rotate , img_label , temp
    
    pix = text_area.get(1.0 , END)
    pix = pix.split('\t')
    pix = np.asarray(pix)
    pix = rotate(pix)
    pix = pix.astype(float)
    plt.imsave('Images/new.png', pix)

    
    def_img = Image.open("Images/new.png")
    re_img = def_img.resize((140,140) , Image.ANTIALIAS)
    im = ImageTk.PhotoImage(re_img)
    img_label.config(image = im,   width=140 , height = 140)


    

In [13]:
def rotate(image):
    image = image.reshape([28, 28])
    image = np.fliplr(image)
    image = np.rot90(image)
    return image

In [14]:
root = Tk()
ico = Image.open('Images/ocr.png')
photo = ImageTk.PhotoImage(ico)
root.wm_iconphoto(False, photo)
root.title("Handwritten English Character Recognition")
root.resizable(0,0)
root.geometry("196x400")

lastx, lasty = None, None
image_number = 0
pred_text = "Predicted: "

im = ImageTk.PhotoImage(Image.open('Images/new.png'))
### for importing

text_area = Text(width=10 , height=1)
text_area.pack()

btn_enter = Button(text="Get Image", fg = 'white' , bg ='black' , command=get_img_from_pixels)
btn_enter.pack()


img_label = Label(image= im , width = 140 , height=140)
img_label.pack()

# image= co_img , width=140 , height = 140




### For drawing
cv = Canvas(root, width=140, height=140, bg='black')

image1 = PIL.Image.new('L', (140,
140), 'black')
draw = ImageDraw.Draw(image1)

cv.bind('<1>', activate_paint)
cv.pack()

btn_save = Button(text="save", fg = 'white' , bg ='black' , command=save)
btn_clear = Button(text="clear", fg = 'white' , bg ='black'  , command=clear)

btn_save.pack(side= LEFT, padx = 10)
btn_clear.pack(side = RIGHT , padx = 10)

btn_pred_im = Button(text="Predict Img",fg = 'white' , bg ='black',  command= pred_img)
btn_pred_im.pack(side= TOP)

btn_pred = Button(text="Predict Draw",fg = 'white' , bg ='black', command= get_pred_dr)
btn_pred.pack(side= BOTTOM)

label_pred = Label(text=pred_text)
label_pred.pack()

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Eyad\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Eyad\AppData\Local\Temp\ipykernel_18520\3268989877.py", line 6, in pred_img
    pix = rotate(pix)
  File "C:\Users\Eyad\AppData\Local\Temp\ipykernel_18520\3964700577.py", line 2, in rotate
    image = image.reshape([28, 28])
ValueError: cannot reshape array of size 1 into shape (28,28)
